This Juypter Notebook provides apis specific to the publication:

 "Towards Continuous Time-Dependent Tomography: Implementation and Evaluation of Continuous Acquisition Schemes in Electron Tomography"

In [ ]:
import tomobase.tiltschemes as tiltschemes
import numpy as np

# Get tilt angles 
#ts = tiltschemes.Binary(-64, 64, 8) # Binary
#ts = tiltschemes.Incremental(-64, 64, 8) # Incremental
ts = tiltschemes.GRS(-64, 64, 8) # GRS

angles = np.array[ts.get_angles() for i in range(65)] # get angles

In [ ]:
# get phantoms nanocage (also referred to as nanoshell - simulate tilt series)

import tomobase.phantoms as phantoms
import tomobase.processes as processes

vol = phantoms.get_nanocage()
sino = processes.project(vol, angles)




In [ ]:
sino_noisy = processes.poisson_noise(sino, 0.1) # 0.1, 1, 10, 100
sino_blurry = processes.gaussian_filter(sino_noisy, 1) 


In [ ]:

rec = processes.optomo_reconstruct(sino, 100, True, False) # Astra. Optomo reconstruct (SIRT, iterations=100)
# weighted by projection is set to false

In [ ]:
from skimage.metrics import structural_similarity

ssim = structural_similarity(vol, rec, data_range=rec.data.max() - rec.data.min())
rmse = np.sqrt(np.mean((vol - rec) ** 2))

print(f'SSIM: {ssim}, RMSE: {rmse}')

In [ ]:
# Artificially create backlash

backlash = 1
indices = np.where(np.diff(sino.angles) < 0)[0] + 1
cumulated_backlash = backlash * len(indices) # total cumullated backlash off entire tilt series
sino.angles[indices] -= backlash




In [ ]:

# Calculating missing wedge
# you can also just generate a tilt series with less projections

sino.sort(True) # sort angles by time
n = 10 # use first 10 angles
angles = sino.angles[:n]
missing_wedge = 180 - (np.max(angles) - np.min(angles))

In [ ]:
# For real data

sino.from_file('path/to/your/data/file')
sino.sort(False) # sort by angle

sino = processes.align_sinogram_center_of_mass(sino)
sino = processes.align_sinogram_xcorr(sino)


In [ ]:
# repeat on loop to align the tilt axis - visualize as you go

sino = processes.align_tilt_axis_shift(sino)
sino = processes.align_tilt_axis_rotation(sino)

In [ ]:
# for comparison to the tilt series reproject and segment
# otsu threshold from skimage
# binary dilate

from skimage.filters import threshold_otsu
from skimage.morphology import binary_dilation

thresh = threshold_otsu(sino.data)
mask = sino.data > thresh
dilated_mask = binary_dilation(mask, iterations=4)

reprojection = processes.project(rec, sino.angles)
reprojection.data = reprojection.data * dilated_mask
sino.data = sino.data * dilated_mask

ssim = structural_similarity(sino.data, reprojection.data, data_range=np.max(reprojection.data) - np.min(reprojection.data))
rmse = np.sqrt(np.mean((sino.data - reprojection.data) ** 2))